## TensorFlow Basics and XOR Problem

In [1]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

ImportError: cannot import name 'pywrap_tensorflow' from partially initialized module 'tensorflow.python' (most likely due to a circular import) (C:\Users\Yunseok Choi\Anaconda3\lib\site-packages\tensorflow\python\__init__.py)

1. TensorFlow Basic

In [ ]:
print(tf.__version__)
hello=tf.constant('Hello, TensorFlow')
print(hello.numpy())
tf.print(hello)

In [ ]:
# Random number generation with uniform distribution
ru_num = tf.random.uniform([3,2],0,1)
tf.print(ru_num)

In [ ]:
# Random number generation with a normal distribution
rn_num = tf.random.normal([2,2,3],0,1)
tf.print(rn_num)

In [ ]:
def sigmo(x):
    """Sigmoid function"""
    return 1/(1+math.exp(-x))
print(sigmo(-5), math.exp(0))

In [ ]:
x = 1; y = 0
#w = tf.random.normal([1],0,1)
w = -4.6025624
result = sigmo(x*w)
tf.print(w, result)

In [ ]:
# Classification with sigmoid activation function: Case 1
alpha = 0.1
for i in range(1000):
    output = sigmo(x*w)
    error = y - output     # calculating error
    w = w + x*alpha*error  # updating w
    if i % 100 == 99:
        tf.print(f'{i}, {error:.5f}, {output:.5f}, {w}')        

In [ ]:
# Classification with sigmoid activation function: Case 2
x = 0; y = 1; alpha = 0.1
w = tf.random.normal([1],0,1)
for i in range(1000):
    output = sigmo(x*w)
    error = y - output
    w = w + x*alpha*error
    if i % 100 == 99:
        tf.print(i, error, output, w)

In [ ]:
# Classification with sigmoid activation function: Case 3
x = 0; y = 1; alpha = 0.1
w = tf.random.normal([1],0,1)
b = tf.random.normal([1],0,1)

In [ ]:
for i in range(1000):
    output = sigmo(x*w+b)
    error = y - output
    w = w + alpha*error
    b = b + alpha*error
    if i % 100 == 99:
        tf.print(f'{i}, {error:.5f}, {output:.5f}, {w}, {b}')

2. And, Or, and Xor

In [ ]:
# And classification
x = np.array([[1,1],[1,0],[0,1],[0,0]])
#y = np.array([[1],[0],[0],[0]])
y = np.array([1,0,0,0])
alpha = 0.1
w = tf.random.normal([2],0,1)
b = tf.random.normal([1],0,1)

In [ ]:
for i in range(2000):
    error_sum = 0
    for j in range(4):
        output = sigmo(x[j].dot(w)+b)
        error = y[j] - output
        w = w + alpha*error*x[j]
        b = b + alpha*error
        error_sum += error
    if i % 200 == 199:
        print(i, error_sum, w.numpy(), b.numpy())

In [ ]:
for i in range(4):
    print(f'X:{x[i]} Y:{y[i]} Prediction: {sigmo(np.sum(x[i]*w)+b):.5f}')

In [ ]:
# Or classification
x = np.array([[1,1],[1,0],[0,1],[0,0]])
y = np.array([1,1,1,0])
alpha = 0.1
w = tf.random.normal([2],0,1)
b = tf.random.normal([1],0,1)
for i in range(4000):
    error_sum = 0
    for j in range(4):
        output = sigmo(x[j].dot(w)+b)
        error = y[j] - output
        w = w + alpha*error*x[j]
        b = b + alpha*error
        error_sum += error
    if i % 200 == 199:
        print(i, error_sum, w.numpy(), b.numpy())
        
for i in range(4):
    print(f'X:{x[i]} Y:{y[i]} Prediction: {sigmo(np.sum(x[i]*w)+b):.5f}')

In [ ]:
# XOR classification: linear classification is not feasible
x = np.array([[1,1],[1,0],[0,1],[0,0]])
y = np.array([1,0,0,1])
alpha = 0.1
w = tf.random.normal([2],0,1)
b = tf.random.normal([1],0,1)
for i in range(2000):
    error_sum = 0
    for j in range(len(x)):
        output = sigmo(np.dot(x[j],w)+b)
        error = y[j] - output
        w = w + alpha*error*x[j]
        b = b + alpha*error
        error_sum += error
    if i % 200 == 199:
        print(i, error_sum, w.numpy(), b.numpy())
        
for i in range(4):
    print(f'X:{x[i]} Y:{y[i]} Prediction: {sigmo(np.sum(x[i]*w)+b):.5f}')

3. Solving XOR with an FNN: TensorFlow (Sequential API)

In [ ]:
x = np.array([[1,1],[1,0],[0,1],[0,0]])
#y = np.array([[0],[1],[1],[0]])
y = np.array([1,0,0,1])

In [ ]:
tf.keras.backend.clear_session()  # removing keras models kept in graph
xor_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=2, activation='sigmoid', input_shape=(2,)),
#    tf.keras.layers.Dense(units=2, activation='sigmoid', input_shape=(2,)),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
xor_model.summary()

In [ ]:
# Alternative way of making neural networks with Sequential API
tf.keras.backend.clear_session() 
xor_model = keras.Sequential()
xor_model.add(layers.Dense(2, activation="sigmoid", input_shape=(2,)))
xor_model.add(layers.Dense(1, activation="sigmoid"))
xor_model.summary()

In [ ]:
# showing computational graph: install pydot and graphviz packages
keras.utils.plot_model(xor_model)

In [ ]:
xor_model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.15), loss='mse')

In [ ]:
%%time
xor_output = xor_model.fit(x, y, epochs = 2500, batch_size = 1, verbose=0)

In [ ]:
xor_model.predict(x)

In [ ]:
for weight in xor_model.weights:
    print(weight.numpy())

In [ ]:
plt.plot(xor_output.history['loss'])
plt.show()

4. Solving XOR with an FNN: TensorFlow (Functional API)

In [ ]:
inputs = tf.keras.Input(shape=(2,))
dense = layers.Dense(2, activation="sigmoid")
h = dense(inputs)
outputs = layers.Dense(1)(h)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name="XOR_Problem")

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "img/XOR_problem_model.png", show_shapes=True)

In [ ]:
model.compile(loss='mse', 
              optimizer=tf.keras.optimizers.SGD(lr=0.15),
             metrics = ["accuracy"])
history_xor = model.fit(x, y, epochs = 1500, batch_size = 1, verbose=0)

In [ ]:
model.predict(x)

In [ ]:
plt.plot(history_xor.history['loss'])
plt.show()

In [ ]:
test_scores = model.evaluate(x, y)
print("Test Loss:", test_scores[0])
print("Test accuracy:", test_scores[1])